![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/4bd64c1deb3c4ffb8ca7ff93c1f1497d).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np

In [75]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTest_carListings.zip', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [5]:
# Visualización datos de test
dataTesting.head()

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand


In [6]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
np.random.seed(42)
y_pred = pd.DataFrame(np.random.rand(dataTesting.shape[0]) * 75000 + 5000, index=dataTesting.index, columns=['Price'])

In [7]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred.to_csv('test_submission.csv', index_label='ID')
y_pred.head()

,Price
ID,
0,33090.508914
1,76303.572981
2,59899.545636
3,49899.386315
4,16701.398033


In [8]:
dataTraining.describe()

,Price,Year,Mileage
count,400000.000000,400000.000000,4.000000e+05
mean,21146.919312,2013.198125,5.507296e+04
std,10753.664940,3.292326,4.088102e+04
min,5001.000000,1997.000000,5.000000e+00
25%,13499.000000,2012.000000,2.584100e+04
50%,18450.000000,2014.000000,4.295500e+04
75%,26999.000000,2016.000000,7.743300e+04
max,79999.000000,2018.000000,2.457832e+06


In [52]:
def dumiesdf(df, category):
    for cat in category:
        cat_dummies = pd.get_dummies(df[cat], prefix=cat)
        df = df.join(cat_dummies)
    df = df.drop(category, axis = 1)
    return(df)

In [68]:
    print("La columna ", col, " es del tipo: ",dataTraining[col].dtype)
    print("En la columna ", col, " Hay: ",sum(dataTraining[col].isna()), " valores vacios.")
    print("En la columna ", col, " Hay: ",len(dataTraining[col].unique()), " valores únicos.")

La columna  State  es del tipo:  int64
En la columna  State  Hay:  0  valores vacios.
En la columna  State  Hay:  51  valores únicos.


In [35]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor


In [62]:
def comparacion_modelos(df, modelos):
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = pd.factorize(df[col])[0]
            
    for modelo in modelos.keys():
        print(modelo, ": ", cross_val_score(modelos[modelo], X = df.drop('Price', axis=1), y = df.Price, cv=10, scoring='neg_mean_squared_error').mean())

In [66]:
modelos = {"Árbol de Decisión": DecisionTreeRegressor(), "Bagging":BaggingRegressor(),
          "Random Forest": RandomForestRegressor(), "AdaBoost":AdaBoostRegressor(),
          "Gradient Boosting": GradientBoostingRegressor(), "XGBoost":XGBRegressor()}

In [67]:
comparacion_modelos(dataTraining, modelos)

Árbol de Decisión :  -24095300.26149493
Bagging :  -15345859.31545955
Random Forest :  -14418450.983738422
AdaBoost :  -115232350.61175816
Gradient Boosting :  -42359766.23914789
XGBoost :  -14604094.463363338


In [78]:
def comparacion_modelos2(df, modelos):
    df = pd.get_dummies(df,drop_first=True)
            
    for modelo in modelos.keys():
        print(modelo, ": ", cross_val_score(modelos[modelo], X = df.drop('Price', axis=1), y = df.Price, cv=10, scoring='neg_mean_squared_error').mean())


In [79]:
comparacion_modelos2(dataTraining, modelos)

Árbol de Decisión :  -22629364.046456467
Bagging :  -15153027.909376765
Random Forest :  -14329818.674668798
AdaBoost :  -123180935.5261786
Gradient Boosting :  -44947832.36702183
XGBoost :  -18491934.89884334
